# <center> <font style="color:rgb(100,109,254)">   Creating a Virtual Pen & Eraser </font> </center>


In [1]:
import cv2
import numpy as np
import time

In [2]:
cap, frame = None, None
# Intializing the webcam feed.
cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)

True

## <font style="color:rgb(134,19,348)">Step 2: Maximizing the Detection Mask and Getting rid of the noise   </font>

In [3]:
# This variable determines if we want to load color range from memory or use the ones defined here. 
load_from_disk = True

# If true then load color range from memory
if load_from_disk:
    penval = np.load('penval.npy')

# Creating A 5x5 kernel for morphological operations
kernel = np.ones((5,5),np.uint8)

In [4]:
def create_object_stack_image():
    global frame
    
    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # If you're reading from memory then load the upper and lower ranges from there
    if load_from_disk:
            lower_range = penval[0]
            upper_range = penval[1]
            
    # Otherwise define your own custom values for upper and lower range.
    else:             
        lower_range  = np.array([26,80,147])
        upper_range = np.array([81,255,255])
    
    mask = cv2.inRange(hsv, lower_range, upper_range)
    
    # Perform the morphological operations to get rid of the noise.
    # Erosion Eats away the white part while dilation expands it.
    mask = cv2.erode(mask,kernel,iterations = 1)
    mask = cv2.dilate(mask,kernel,iterations = 2)

    # Visualize the real part of the target color (Optional)
    res = cv2.bitwise_and(frame,frame, mask= mask)
    
    #  Converting the binary mask to 3 channel image, this is just so we can stack it with the others
    mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    
    # stack all frames and show it
    obj_stacked_img = np.hstack((mask, frame, res))
    
    return obj_stacked_img

In [5]:
while True:
    
    ret, frame = cap.read()
    if not ret:
        break
        
    frame = cv2.flip(frame, 1)

    obj_stacked_img = create_object_stack_image()
    cv2.imshow('Trackbars',cv2.resize(obj_stacked_img, None, fx=0.4, fy=0.4))
    
    # If the user presses ESC then exit the program
    key = cv2.waitKey(1)
    if key == 27:
        break

cv2.destroyAllWindows()
cap.release()